In [ ]:
import numpy as np
from sympy import Matrix

In [ ]:
# Caesar Cipher

def caesar_encrypt(text, key):
    result = ""
    for char in text:
        if char.isalpha():
            shift = (ord(char.upper()) - 65 + key) % 26
            result += chr(shift + 65)
        else:
            result += char
    return result

def caesar_decrypt(text, key):
    return caesar_encrypt(text, -key)

In [ ]:
# Affine Cipher

def modinv(a, m):
    a = a % m
    for x in range(1, m):
        if (a * x) % m == 1:
            return x
    raise ValueError("No modular inverse")

def affine_encrypt(text, a, b):
    result = ""
    for char in text:
        if char.isalpha():
            result += chr(((a*(ord(char.upper())-65)+b)%26)+65)
        else:
            result += char
    return result

def affine_decrypt(text, a, b):
    a_inv = modinv(a, 26)
    result = ""
    for char in text:
        if char.isalpha():
            result += chr((a_inv*((ord(char.upper())-65)-b)%26)+65)
        else:
            result += char
    return result

In [ ]:
# Playfair Cipher

def playfair_matrix(key):
    key = "".join(dict.fromkeys(key.upper().replace("J","I")))
    matrix = []
    for c in key:
        if c not in matrix:
            matrix.append(c)
    for c in "ABCDEFGHIKLMNOPQRSTUVWXYZ":
        if c not in matrix:
            matrix.append(c)
    return [matrix[i:i+5] for i in range(0, 25, 5)]

def playfair_encrypt(text, key):
    matrix = playfair_matrix(key)
    text = text.upper().replace("J","I").replace(" ", "")
    # prepare digraphs
    i = 0
    digraphs = []
    while i < len(text):
        a = text[i]
        b = ''
        if i+1 < len(text):
            b = text[i+1]
            if a == b:
                b = 'X'
                i += 1
            else:
                i += 2
        else:
            b = 'X'
            i += 1
        digraphs.append(a+b)
    result = ""
    for a,b in digraphs:
        ax, ay, bx, by = -1,-1,-1,-1
        for row in range(5):
            for col in range(5):
                if matrix[row][col]==a:
                    ax, ay = row, col
                if matrix[row][col]==b:
                    bx, by = row, col
        if ax==bx:
            result += matrix[ax][(ay+1)%5] + matrix[bx][(by+1)%5]
        elif ay==by:
            result += matrix[(ax+1)%5][ay] + matrix[(bx+1)%5][by]
        else:
            result += matrix[ax][by] + matrix[bx][ay]
    return result

def playfair_decrypt(text, key):
    matrix = playfair_matrix(key)
    result = ""
    for i in range(0,len(text),2):
        a,b = text[i], text[i+1]
        ax, ay, bx, by = -1,-1,-1,-1
        for row in range(5):
            for col in range(5):
                if matrix[row][col]==a:
                    ax, ay = row, col
                if matrix[row][col]==b:
                    bx, by = row, col
        if ax==bx:
            result += matrix[ax][(ay-1)%5] + matrix[bx][(by-1)%5]
        elif ay==by:
            result += matrix[(ax-1)%5][ay] + matrix[(bx-1)%5][by]
        else:
            result += matrix[ax][by] + matrix[bx][ay]
    return result

In [ ]:
# Hill Cipher (2x2)

def hill_encrypt(text, key_matrix):
    text = text.upper().replace(" ", "")
    if len(text) % 2 != 0:
        text += "X"
    result = ""
    for i in range(0, len(text), 2):
        vec = np.array([[ord(text[i])-65],[ord(text[i+1])-65]])
        enc = np.dot(key_matrix, vec) % 26
        result += chr(enc[0,0]+65) + chr(enc[1,0]+65)
    return result

def hill_decrypt(text, key_matrix):
    text = text.upper().replace(" ", "")
    det = int(np.round(np.linalg.det(key_matrix))) % 26
    det_inv = modinv(det, 26)
    key_inv = det_inv * np.round(det * np.linalg.inv(key_matrix)).astype(int) % 26
    result = ""
    for i in range(0, len(text), 2):
        vec = np.array([[ord(text[i])-65],[ord(text[i+1])-65]])
        dec = np.dot(key_inv, vec) % 26
        result += chr(int(dec[0,0])+65) + chr(int(dec[1,0])+65)
    return result

In [ ]:
# Main Interface

def main():
    while True:
        print("\nClassic Cipher Tool")
        print("1. Caesar Cipher")
        print("2. Affine Cipher")
        print("3. Playfair Cipher")
        print("4. Hill Cipher (2x2)")
        print("5. Exit")
        choice = input("Select Cipher: ")

        if choice=='5':
            break

        operation = input("Encrypt or Decrypt (E/D): ").upper()
        text = input("Enter text: ")

        if choice=='1':
            key = int(input("Enter key (integer): "))
            if operation=='E':
                print("Ciphertext:", caesar_encrypt(text,key))
            else:
                print("Plaintext:", caesar_decrypt(text,key))

        elif choice=='2':
            a = int(input("Enter key a (coprime with 26): "))
            b = int(input("Enter key b: "))
            if operation=='E':
                print("Ciphertext:", affine_encrypt(text,a,b))
            else:
                print("Plaintext:", affine_decrypt(text,a,b))

        elif choice=='3':
            key = input("Enter keyword: ")
            if operation=='E':
                print("Ciphertext:", playfair_encrypt(text,key))
            else:
                print("Plaintext:", playfair_decrypt(text,key))

        elif choice=='4':
            key_matrix = []
            print("Enter 2x2 key matrix row-wise (integers 0-25):")
            for i in range(2):
                row = list(map(int,input(f"Row {i+1}: ").split()))
                key_matrix.append(row)
            key_matrix = np.array(key_matrix)
            if operation=='E':
                print("Ciphertext:", hill_encrypt(text,key_matrix))
            else:
                print("Plaintext:", hill_decrypt(text,key_matrix))
        else:
            print("Invalid choice!")

if __name__=="__main__":
    main()


Classic Cipher Tool
1. Caesar Cipher
2. Affine Cipher
3. Playfair Cipher
4. Hill Cipher (2x2)
5. Exit
Select Cipher: 1
Encrypt or Decrypt (E/D): e
Enter text: HELLO WORLD
Enter key (integer): 3
Ciphertext: KHOOR ZRUOG

Classic Cipher Tool
1. Caesar Cipher
2. Affine Cipher
3. Playfair Cipher
4. Hill Cipher (2x2)
5. Exit
Select Cipher: 1
Encrypt or Decrypt (E/D): d
Enter text: KHOOR ZRUOG
Enter key (integer): 3
Plaintext: HELLO WORLD

Classic Cipher Tool
1. Caesar Cipher
2. Affine Cipher
3. Playfair Cipher
4. Hill Cipher (2x2)
5. Exit
Select Cipher: 2
Encrypt or Decrypt (E/D): e
Enter text: CRYPTOGRAPHY
Enter key a (coprime with 26): 5
Enter key b: 8
Ciphertext: SPYFZAMPIFRY

Classic Cipher Tool
1. Caesar Cipher
2. Affine Cipher
3. Playfair Cipher
4. Hill Cipher (2x2)
5. Exit
Select Cipher: 2
Encrypt or Decrypt (E/D): d
Enter text: SPYFZAMPIFRY
Enter key a (coprime with 26): 5
Enter key b: 8
Plaintext: CRYPTOGRAPHY

Classic Cipher Tool
1. Caesar Cipher
2. Affine Cipher
3. Playfair Ciphe